## Adding a subset of COCO people data

In [1]:
import glob, json, os
import numpy as np
import cv2
import pycocotools.mask as mask_util
from PIL import Image, ImageDraw
from copy import deepcopy

from luxonis_ml.data import *

### Download and extract data

In [2]:
! pip install gdown
! gdown 1XlvFK7aRmt8op6-hHkWVKIJQeDtOwoRT -O ../data/COCO_people_subset.zip
! unzip ../data/COCO_people_subset.zip -d ../data/

  Using cached gdown-4.7.1-py3-none-any.whl (15 kB)
  Using cached beautifulsoup4-4.12.2-py3-none-any.whl (142 kB)
  Using cached soupsieve-2.4.1-py3-none-any.whl (36 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Downloading...
From: https://drive.google.com/uc?id=1XlvFK7aRmt8op6-hHkWVKIJQeDtOwoRT
To: /home/klemen/luxonis/luxonis-ml/data/COCO_people_subset.zip
100%|██████████████████████████████████████| 7.78M/7.78M [00:00<00:00, 55.5MB/s]
Archive:  ../data/COCO_people_subset.zip
replace ../data/person_keypoints_val2017.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


### Find image paths and load COCO annotations

In [3]:
img_dir = '../data/person_val2017_subset'
annot_file = '../data/person_keypoints_val2017.json'

# get paths to images sorted by number
im_paths = glob.glob(img_dir+'/*.jpg')
nums = np.array([int(path.split('/')[-1].split('.')[0]) for path in im_paths])
idxs = np.argsort(nums)
im_paths = list(np.array(im_paths)[idxs])

# load 
with open(annot_file) as file:
    data = json.load(file)
imgs = data['images']
anns = data['annotations']

### Convert from COCO format

The LDF will expect the following format:
```
additions (list[dict]): {
    # path to the image on local storage
    'filepath' (str): "my_image.jpg",
    # list of boxes, which are length 6 lists of [class, x, y, width, height]
    'boxes' (list[list]): [["apple", 0.1, 0.2, 0.1, 0.2], ...],
    # list of keypoint instances, which are length 2 [class, points] and points is a list of (x,y) tuples
    'keypoints' (list[str|int, list[tuple]]): [["apple", [(0.1, 0.2), (0.3, 0.4), ...], ...],
    # an integer numpy array of HxW where the values correspond to classes
    'segmentation' (np.ndarray): np.zeros((height, width))
}
```

In [4]:
# create some artificial splits
splits = ['train' for _ in range(20)] + ['val' for _ in range(10)]

additions = [] # list of additional training examples

for i, path in enumerate(im_paths):
    # find annotations matching the COCO image
    gran = path.split('/')[-1]
    img = [img for img in imgs if img['file_name']==gran][0]
    img_id = img['id']
    img_anns = [ann for ann in anns if ann['image_id'] == img_id]
    
    # load the image
    im = cv2.imread(path)
    height, width, _ = im.shape
    
    # initialize annotations for LDF
    mask = np.zeros((height, width)) # segmentation mask is always a HxW numpy array
    boxes = [] # bounding boxes are a list of [class, x, y, width, height] of the box
    keypoints = [] # keypoints are a list of classes and (x,y) points
    
    for ann in img_anns:
        # COCO-specific conversion for segmentation
        seg = ann['segmentation']
        if isinstance(seg, list):
            for s in seg:
                poly = np.array(s).reshape(-1,2)
                poly = [(poly[i,0],poly[i,1]) for i in range(len(poly))]
                m = Image.new('L', (width, height), 0)
                ImageDraw.Draw(m).polygon(poly, outline=1, fill=1)
                m = np.array(m)
                mask[m==1] = 1
        
        # COCO-specific conversion for bounding boxes
        x, y, w, h = ann['bbox']
        boxes.append(['person', x/width, y/height, w/width, h/height])
        
        # COCO-specific conversion for keypoints
        kps = np.array(ann['keypoints']).reshape(-1, 3)
        keypoint = []
        for kp in kps:
            if kp[2] == 0:
                keypoint.append((float('nan'), float('nan')))
            else: 
                keypoint.append((kp[0]/width, kp[1]/height))  
        keypoints.append(['person', keypoint])
    
    # dictionary structure expected by LDF
    additions.append({
        'image': {
            'filepath': path,
            'segmentation': mask,
            'boxes': boxes,
            'keypoints': keypoints,
            'split': splits[i]
        }
    })
    
original_additions = deepcopy(additions)

In [5]:
team_id = "luxonis_team_id"
team_name = "luxonis"
dataset_name = "coco_test"

dataset_id = LuxonisDataset.create(
    team_id=team_id,
    team_name=team_name,
    dataset_name=dataset_name

)
print(f"Dataset id: {dataset_id}")

Dataset id: 64a3c86e0363aedcae0102d4


In [6]:
with LuxonisDataset(
    team_id=team_id,
    dataset_id=dataset_id,
    team_name=team_name,
    dataset_name=dataset_name,
    bucket_type="local"
) as dataset:
    
    # defines the format of the COCO data source,
    # which in this case is just a single image per training example
    dataset.create_source(
        'coco',
        custom_components=[
            LDFComponent('image', HType.IMAGE, IType.BGR)
        ]
    )
    
    dataset.set_classes(['person']) # needed for classification and detection
    dataset.set_mask_targets({1:'person'}) # needed for segmentation
    dataset.set_skeleton({
        'labels': data['categories'][0]['keypoints'],
        'edges': (np.array(data['categories'][0]['skeleton'])-1).tolist()
    }) # optional for keypoints to define the skeleton visualization
    
    dataset.add(additions)
    dataset.create_version(note="Initially adding data")

Checking for additions or modifications...


100%|██████████| 30/30 [00:03<00:00,  9.82it/s]


Extracting dataset media...


100%|██████████| 30/30 [00:00<00:00, 32.35it/s]


 100% |███████████████████| 30/30 [5.5s elapsed, 0s remaining, 14.0 samples/s]      


In [7]:
with LuxonisDataset(
    team_id=team_id,
    dataset_id=dataset_id,
) as dataset:
    
    additions = deepcopy(original_additions)
    
    # add a classification label to the dataset
    for addition in additions:
        addition['image']['class'] = 'person'
    
    dataset.add(additions)
    dataset.create_version(note="Add classification labels")

Checking for additions or modifications...


100%|██████████| 30/30 [00:08<00:00,  3.62it/s]


Extracting dataset media...


100%|██████████| 30/30 [00:00<00:00, 30.78it/s]


In [8]:
with LuxonisDataset(
    team_id=team_id,
    dataset_id=dataset_id,
) as dataset:
    
    print("Dataset version:", dataset.version)
    v10 = dataset.fo_dataset.load_saved_view('version_1.0')
    print("Class attribute presence in version 1.0:")
    print([sample['class'] is not None for sample in v10])
    v11 = dataset.fo_dataset.load_saved_view('version_1.1')
    print("Class attribute presence in version 1.1:")
    print([sample['class'] is not None for sample in v11])

Dataset version: 1.1
Class attribute presence in version 1.0:
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
Class attribute presence in version 1.1:
[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


### Permanently delete dataset

In [9]:
with LuxonisDataset(
    team_id=team_id,
    dataset_id=dataset_id,
) as dataset:
    
    dataset.delete_dataset()